# RSS Article Batch Download Examples

This notebook demonstrates how to batch download RSS articles from S3.

In [1]:
import os
import pandas as pd
from time import time


In [2]:
# Root imports
os.chdir('..')
print(os.getcwd())

from src.search.batch import S3BatchDownloader
from dotenv import load_dotenv

load_dotenv(override=True)

downloader = S3BatchDownloader()

os.chdir('example-notebooks')


/home/ceg98/IngestRSS


## Initialize the Downloader

In [3]:
import boto3
from botocore.exceptions import ClientError

s3 = boto3.client('s3')

# Test ListObjects
try:
    response = s3.list_objects_v2(Bucket=os.getenv("S3_BUCKET_NAME"))
    print("List permission: Allowed")
except ClientError as e:
    print("List permission: Denied")


List permission: Allowed


In [4]:
start = time()
output_path = "consolidated_data.csv"  # or "consolidated_data.json"

# Define date range
start_date = "2024-11-17" # FIXME: Fix the error where data can't be collected before the date you started collecting.
end_date = "2024-11-22"

# Start downloading
downloader.download_to_file(
    output_path=output_path,
    file_format="csv",  # or "json"
    start_date=start_date,
    end_date=end_date
)

print(f"Downloaded data to {output_path} in {time() - start:.2f} seconds")

Found 59592 objects to process


100%|██████████| 59592/59592 [06:23<00:00, 155.31object/s]


Downloaded data to consolidated_data.csv in 463.50 seconds


In [5]:
df = pd.read_csv(output_path)

In [6]:
df.head()

,link,rss,title,content,unixTime,rss_id,article_id,llm_summary,embedding,unixtime
0,https://meetings.skift.com/2024/11/07/critical...,https://www.eventmanagerblog.com/feed,Critical Accessibility Gaps Exist in Business ...,Joint research assesses the current state of i...,1731883626,22219df62c,0236fb6aab,NaN,NaN,1731883626
1,https://www.npr.org/2021/08/27/1031717918/plan...,https://www.npr.org/rss/podcast.php?id=510289,Planet Money Summer School 6: Crypto & Commenc...,Planet Money Summer School 6: Crypto & Commenc...,1731884703,8181d7a585,03c9f1092b,NaN,NaN,1731884703
2,https://www.ufc.com/news/mauricio-ruffy-enjoyi...,https://www.ufc.com/rss/news,Mauricio Ruffy Is Enjoying The Process,"“I saw the UFC's care for me, always putting m...",1731883664,2c774cd014,05cfd6979f,NaN,NaN,1731883664
3,https://www.vg247.com/dragon-age-the-veilguard...,https://www.vg247.com/feed/,Dragon Age: The Veilguard's first patch isn't ...,Dragon Age: The Veilguard has now gotten its f...,1731883714,374d98cbb9,00505e505b,NaN,NaN,1731883714
4,https://www.frugalrules.com/finances-back-on-t...,https://www.frugalrules.com/feed/,How to Get Your Finances Back on Track Startin...,"We all face it. Life gets the best of us, and ...",1731883633,e18025b98e,03c3406d27,NaN,NaN,1731883633


# Cleaning Data - EDA ( Delete Later ) 

In [7]:
df = df.dropna(subset=['content'])

In [8]:
df.shape

(58273, 10)

In [9]:
df.sample(20)["content"].values

array(['Let’s face it. The world of team building activities has evolved beyond trust falls and awkward ice breakers. CEOs, entrepreneurs, professors—you name it—are constantly on the hunt for dynamic ways to strengthen their teams. Why? Because a connected team is the powerhouse behind every great achievement. Imagine transforming mundane meetings into epic adventures or dull Fridays into days buzzing with excitement and laughter. With the right mix, you can catapult your group from mere co-workers to a unified force ready to tackle any challenge head-on. Ready to get started? Let’s take a look at some team building activities that you might want to use.\n\nThe Importance of Team Building for Company Culture\n\nGone are the days when “team building” was merely a buzzword. Nowadays, it’s essential for nurturing company culture and ensuring everyone feels like part of something bigger than their task list. These activities are all about boosting morale, enhancing communication, and fost

In [ ]:
def pinecone_save_article(article:dict):
    logger.info("Saving article to Pinecone")
    index = get_index()

    # Expected Keys from Pinecone *MUST* include 'id' and 'values'
    article["id"] = article["article_id"]
    article["values"] = vectorize(article["content"])
    
    namespace = f"IngestRSS-Articles"
    
    logger.info("Upserting article to Pinecone")
    upsert_vectors(index, [article], namespace)
    logger.info(f"Successfully upserted article w/ article-id: {article["article_id"]} to Pinecone index {index.name} with namespace {namespace}")

SyntaxError: f-string: unmatched '[' (3638322678.py, line 13)

: 